# TensorFlow Hub ile Transfer Öğrenme

[TensorFlow Hub](https://tfhub.dev/), önceden eğitilmiş TensorFlow modellerinin bir deposudur.

Bu çalışma dosyasında aşağıdakilerin nasıl yapılacağı gösterilmektedir:

1. TensorFlow Hub'dan `tf.keras` içeren modelleri kullanın.
1. TensorFlow Hub'dan bir görüntü sınıflandırma modeli kullanın.
1. Kendi görüntü sınıflarınız için bir modele ince ayar yapmak için basit transfer öğrenimi yapın.

## Kütüphaneleri Tanımlama

In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard

## Bir ImageNet sınıflandırıcı

[ImageNet](https://en.wikipedia.org/wiki/ImageNet) kıyaslama veri setinde önceden eğitilmiş bir sınıflandırıcı modeli kullanarak başlayacaksınız; başlangıç eğitimi gerekmez!

### Sınıflandırıcıyı indirin

[TensorFlow Hub'dan]((https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2)) önceden eğitilmiş <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> modelini seçin ve [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) ile Keras katmanı olarak tanımlayın. 

<a href="https://tfhub.dev/s?q=tf2&module-type=image-classification/" class="external">Burada</a> açılır menüden sağlanan örnekler de dahil olmak üzere, TensorFlow Hub'dan herhangi bir uyumlu görüntü sınıflandırıcı modeli burada çalışacaktır.

In [ ]:
mobilenet_v2 ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

In [ ]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

### Tek bir görüntü üzerinde çalıştırın

Modeli denemek için tek bir görsel indirin:

In [ ]:
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
grace_hopper

In [ ]:
grace_hopper = np.array(grace_hopper)/255.0
grace_hopper.shape

Bir batch boyutu ekleyin (`np.newaxis` ile) ve resmi modele iletin:

In [ ]:
result = classifier.predict(grace_hopper[np.newaxis, ...])
result.shape

Sonuç, görüntü için her sınıfın olasılığını derecelendiren 1001 elemanlı bir logit vektörüdür.

En üst sınıf kimliği `tf.math.argmax` ile bulunabilir:

In [ ]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class

### Tahminlerin kodunu çöz

`predicted_class` kimliğini (`653` gibi) alın ve tahminlerin kodunu çözmek için ImageNet veri kümesi etiketlerini alın:

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
plt.imshow(grace_hopper)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

## Basit transfer öğrenimi

Ancak, orijinal ImageNet veri kümesinde yer almayan (önceden eğitilmiş modelin eğitildiği) sınıfları olan kendi veri kümenizi kullanarak özel bir sınıflandırıcı oluşturmak isterseniz ne olur?

Bunu yapmak için şunları yapabilirsiniz:

1. TensorFlow Hub'dan önceden eğitilmiş bir model seçin; ve
2. Özel veri kümenizdeki sınıfları tanımak için üst (son) katmanı yeniden eğitin.

### Veri Kümesi

Bu örnekte, TensorFlow çiçekler veri kümesini kullanacaksınız:

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos',
  'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

İlk olarak, bir `tf.data.Dataset` oluşturacak olan `tf.keras.utils.image_dataset_from_directory` ile disk dışındaki görüntü verilerini kullanarak bu verileri modele yükleyin:

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
  str(data_root),
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

Çiçekler veri kümesinin beş sınıfı vardır:

In [ ]:
class_names = np.array(train_ds.class_names)
print(class_names)

İkincisi, TensorFlow Hub'ın görüntü modelleri için kuralı `[0, 1]` aralığında kayan girişler beklemek olduğundan, bunu başarmak için `tf.keras.layers.Rescaling` ön işleme katmanını kullanın.

Not: Modelin içine `tf.keras.layers.Rescaling` katmanını da dahil edebilirsiniz. Dengelerin tartışılması için [Ön işleme katmanlarıyla çalışma](https://www.tensorflow.org/guide/keras/preprocessing_layers) kılavuzuna bakın.

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

Üçüncü olarak, `Dataset.prefetch` ile arabelleğe alınmış önceden getirmeyi kullanarak giriş hattını tamamlayın, böylece verileri I/O engelleme sorunları olmadan diskten alabilirsiniz.

Bunlar, veri yüklerken kullanmanız gereken en önemli `tf.data` yöntemlerinden bazılarıdır. İlgilenen okuyucular, [tf.data API ile daha iyi performans](https://www.tensorflow.org/guide/data_performance#prefetching) bölümünden verilerin diske nasıl önbelleğe alınacağını ve diğer tekniklerin yanı sıra bunlar hakkında daha fazla bilgi edinebilir. kılavuz.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

### Sınıflandırıcıyı bir dizi görüntü üzerinde çalıştırın

Şimdi sınıflandırıcıyı toplu(batch) bir görüntü olarak çalıştırın:

In [ ]:
result_batch = classifier.predict(train_ds)

In [ ]:
predicted_class_names = imagenet_labels[tf.math.argmax(result_batch, axis=-1)]
predicted_class_names

Bu tahminlerin resimlerle nasıl örtüştüğünü kontrol edin:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')
_ = plt.suptitle("ImageNet predictions")

Sonuçlar mükemmel olmaktan uzak, ancak bunların modelin eğitildiği sınıflar olmadığı düşünüldüğünde ("papatya (daisy)" hariç) makul.

### Sınıflandırma Katmanı Olmayan Model İndirin

TensorFlow Hub, üst sınıflandırma katmanı olmayan modelleri de dağıtır. Bunlar, transfer öğrenmeyi kolayca gerçekleştirmek için kullanılabilir.

<a href="https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4" class="external">TensorFlow Hub'dan</a> <a href="https://arxiv.org/abs/1801.04381" class="external">MobileNetV2</a> önceden eğitilmiş bir model seçin. Açılır menüdeki örnekler de dahil olmak üzere TensorFlow Hub'dan herhangi bir uyumlu <a href="https://tfhub.dev/s?module-type=image-feature-vector&q=tf2" class="external">görüntü özelliği vektör modeli</a> burada çalışacaktır.

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = mobilenet_v2 #@param ["mobilenet_v2", "inception_v3"] {type:"raw"}

Önceden eğitilmiş modeli bir Keras katmanı olarak [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) ile özellik çıkarıcıyı oluşturun. Değişkenleri dondurmak için `trainable=False` bağımsız değişkenini kullanın, böylece eğitim yalnızca yeni sınıflandırıcı katmanını değiştirir:

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

Öznitelik çıkarıcı, her görüntü için 1280 uzunluğunda bir vektör döndürür (bu örnekte görüntü batch boyutu 32'de kalır):

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

### Modele bir sınıflandırma katmanı ekleyin

Modeli tamamlamak için, özellik çıkarıcı katmanını bir `tf.keras.Sequential` modeline sarın ve sınıflandırma için tam bağlantılı bir katman ekleyin:

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
predictions = model(image_batch)

In [ ]:
predictions.shape

### Modeli eğitin

Eğitim sürecini yapılandırmak için `Model.compile` kullanın ve log oluşturmak ve depolamak için bir `tf.keras.callbacks.TensorBoard` geri çağrısı (callback) ekleyin:

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

Şimdi modeli eğitmek için `Model.fit` yöntemini kullanın.

Bu örneği kısa tutmak için sadece 10 devir(epochs) için eğitim alacaksınız. TensorBoard'daki eğitim ilerlemesini daha sonra görselleştirmek için, günlükleri bir [TensorBoard geri çağrısı(callback)](https://www.tensorflow.org/tensorboard/get_started#using_tensorboard_with_keras_modelfit) oluşturun ve saklayın.


In [ ]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Metriklerin her devirde(epoch'da) nasıl değiştiğini görmek ve diğer skaler değerleri izlemek için TensorBoard'u başlatın:

In [ ]:
%tensorboard --logdir logs/fit

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/images/tensorboard_transfer_learning_with_hub.png?raw=1"/> -->

### Tahminleri kontrol edin

Model tahminlerinden sıralı sınıf adları listesini alın:

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

Model tahminlerini çizin:

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Modelinizi dışa aktarın ve yeniden yükleyin

Artık modeli eğittiğinize göre, daha sonra yeniden kullanmak üzere SavedModel olarak dışa aktarın.

In [ ]:
t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path)

export_path

SavedModel'i yeniden yükleyebileceğinizi ve modelin aynı sonuçları verebileceğini doğrulayın:

In [ ]:
reloaded = tf.keras.models.load_model(export_path)

In [ ]:
result_batch = model.predict(image_batch)
reloaded_result_batch = reloaded.predict(image_batch)

In [ ]:
abs(reloaded_result_batch - result_batch).max()

In [ ]:
reloaded_predicted_id = tf.math.argmax(reloaded_result_batch, axis=-1)
reloaded_predicted_label_batch = class_names[reloaded_predicted_id]
print(reloaded_predicted_label_batch)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(reloaded_predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

## Sonraki adımlar

Çıkarım(Inference) için modeli yüklemek veya bir [TensorFlow Lite](https://www.tensorflow.org/lite/convert/) modeline (cihazda makine öğrenimi için) veya bir [TensorFlow.js](https://www.tensorflow.org/js/tutorials#convert_pretrained_models_to_tensorflowjs) modeline (JavaScript'te makine öğrenimi için) dönüştürmek için SavedModel'i kullanabilirsiniz.

TensorFlow Hub'ın önceden eğitilmiş modellerini resim, metin, ses ve video görevlerinde nasıl kullanacağınızı öğrenmek için [daha fazla eğitici notları](https://www.tensorflow.org/hub/tutorials) keşfedin.